LRU 算法

https://labuladong.github.io/article/fname.html?fname=LRU算法

In [ ]:
# 146. LRU 缓存
# https://leetcode.cn/problems/lru-cache/

class LRUCache:

    def __init__(self, capacity: int):
        pass

    def get(self, key: int) -> int:
        pass

    def put(self, key: int, value: int) -> None:
        pass


# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)